In [1]:
import re
import pandas as pd
from pandas import concat

import warnings

from helper_functions.write_excel_file import write_excel_file


def parse_pin_file(file_path):
    # Additional info
    additional_info = {}
    additional_info['McuName'] = ''
    additional_info["McuPackage"] = ''
    additional_info["McuCPN"] = ''
    
    # Define column names
    columns = ['PinNumber', 'PinName', 'Mode', 'Signal']

    # Create an empty DataFrame with the specified columns
    pin_data = pd.DataFrame(columns=columns)
    current_pin_number = None
    current_pin_name = None

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Mcu.Name"):
            additional_info['McuName'] = (line.split("=")[1]).split("\n")[0]
        elif line.startswith("Mcu.Package"):
            additional_info["McuPackage"] = (line.split("=")[1]).split("\n")[0]
        elif line.startswith("Mcu.CPN"):
            additional_info["McuCPN"] = (line.split("=")[1]).split("\n")[0]

        elif line.startswith("Mcu.Pin"):
            pin_match = re.match(r'Mcu\.Pin(\d+)=(\w+)', line)
            if pin_match:
                current_pin_number, current_pin_name = pin_match.groups() if pin_match else (None, None)
                current_pin_number = int(current_pin_number)
                # Add a new row
                new_data = {'PinNumber': current_pin_number, 'PinName': current_pin_name, 'Mode': '', 'Signal': ''}
                pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
                #pin_data[current_pin_number] = {'PinName': current_pin_name}
        elif "RCC_OSC_IN" in line:
            new_data = {'PinNumber': '', 'PinName': line.split("-")[0], 'Mode': '', 'Signal': "RCC_OSC_IN"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
        elif "RCC_OSC_OUT" in line:
            new_data = {'PinNumber': '', 'PinName': line.split("-")[0], 'Mode': '', 'Signal': "RCC_OSC_OUT"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
        elif "RCC_OSC32_IN" in line:
            new_data = {'PinNumber': '', 'PinName': line.split("-")[0], 'Mode': '', 'Signal': "RCC_OSC32_IN"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
        elif "RCC_OSC32_OUT" in line:
            new_data = {'PinNumber': '', 'PinName': line.split("-")[0], 'Mode': '', 'Signal': "RCC_OSC32_OUT"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
    for line in lines:
        #if current_pin_name is not None and line.startswith(current_pin_name):
        for pin_name in pin_data['PinName']:
            mode_match = re.match(r'{}\.(Mode)=(\w+)'.format(pin_name), line)
            signal_match = re.match(r'{}\.(Signal)=(\w+)'.format(pin_name), line)
            if not mode_match and not signal_match:
                continue
            match = mode_match if mode_match else signal_match
            i = pin_data[pin_data['PinName'] == pin_name].index[0]
            pin_data.loc[i, 'Mode' if mode_match else 'Signal'] = match.group(2)

    # combine the data of the pins which are RCC_ into one line
    pin_data = pin_data.groupby('PinName').agg({'PinNumber': 'first', 'Mode': 'first', 'Signal': ''.join}).reset_index()

    # check that each EXTI line has only been selected once
    # Check for duplicates in the "Signal" column
    duplicates = pin_data[pin_data['Signal'].str.contains('GPXTI1') & pin_data.duplicated(subset='Signal', keep=False)]

    duplicate_EXTI_error = False

    # If there are duplicates, raise a warning
    if not duplicates.empty:
        duplicate_EXTI_error = True
        warning_message = "Warning: Duplicate values found in the 'Signal' column."
        warnings.warn(warning_message)
        print(duplicates)


    return pin_data, additional_info, duplicate_EXTI_error




if __name__ == "__main__":
    file_path = "STM32F722IET7_Pinout.ioc"  # Replace with the actual path to your file

    pin_data, additional_info, duplicate_EXTI_error = parse_pin_file(file_path)


    # Display the resulting DataFrame
    #print(pin_data)

    # Specify the Excel file path
    excel_file_path = file_path[:-3]+"xlsx"

    write_excel_file(excel_file_path, pin_data, additional_info, duplicate_EXTI_error)


DataFrame exported to STM32F722IET7_Pinout.xlsx with alternating row colors and without the first line.
